In [9]:
import requests
import os
import pandas as pd
from datetime import date
import time
from io import StringIO

hodeCanvas = {"Authorization": f'Bearer {os.environ["tokenCanvas"]}'}
parametreCanvas = {"per_page": 100}
idag = date.today().isoformat()[0:10]
url = "https://hvl.instructure.com/api/v1/accounts/1/sis_imports"
svar = requests.get(url, headers=hodeCanvas, params=parametreCanvas)
importerte_filer = svar.json()



In [10]:
len(importerte_filer['sis_imports'])

100

In [ ]:

for i in importerte_filer['sis_imports']:

    # Her må eg bruke litt ulik logikk for å hente ut ulike filar:
    # Feilmeldingar: "errors_attachment" in i,
    # Ulike vedlegg: "csv_attachments\[accounts|courses|enrollments|sections|terms|users_filtered]" for ulike andre
    # Eg kan også med fordel teste på dato/tid for å finne eventuell masteroppdatering (sjølv om dette vert endra med ny import ...)

    if ("errors_attachment" in i) and (i['created_at'] > idag):
        fil = i["errors_attachment"]['url']
        data = pd.read_csv(StringIO(requests.get(fil, headers=hodeCanvas).content.decode("utf-8")))
        brukarar = data[data['message'].str.startswith('User not found')]
        eksisterer = data[data['message'].str.startswith('An existing')]
        for b in brukarar['message']:
            innhald += f"{b}\n"
            antal_feilmeldingar += 1
        for e in eksisterer['message']:
            innhald += f"{e}\n"
            antal_feilmeldingar += 1
print(innhald)